Reliability Models for Facility Location:The Expected Failure Cost Case
===

Base on the following article:

*Snyder, L. V., & Daskin, M. S. (2005). Reliability models for facility location: the expected failure cost case. Transportation Science, 39(3), 400-416.*

Part III - The Reliability P-Median Problem
---

In [2]:
# Colecting the problem
import sys
sys.path.append('../../PythonLib')

from dataset.mongodb import MongoClient
# Solving the problem
from solvers.uflp import uflp
# Representing the network
from dataviz.network import Network

from sklearn.neighbors import DistanceMetric
from geopy.distance import great_circle

import scipy as sp
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

First, we collect the dataset from our database using **MonClient**.

In [3]:
dataset = MongoClient(db = {
# MongoClient let us connect to the database and manipulate our dataset
    "mongo_host": "ns396089.ip-37-59-38.eu",
    "mongo_port": 32771,
    "mongo_db_name": "NETWORK-AND-DISCRETE-LOCATION"
}, q = {
    'metadata.author': 'Mark S. Daskin',
    'metadata.topic': 'NETWORK-AND-DISCRETE-LOCATION',
    'metadata.dataset': '49-nodes'
},f=None)

We can display the dataset using the method **get** which return a pandas Dataframe containing our data

In [4]:
df = dataset.get()
df.head()

,ID,LONGITUDE,LATITUDE,STATE POP,CITY POP,FIXED COST,CITY
0,1,-121.467360,38.566850,29760021,369365,115800,< Sacramento CA >
1,2,-73.799017,42.665750,17990455,101082,101800,< Albany NY >
2,11,-84.422592,33.762900,6478216,394017,71200,< Atlanta GA >
3,12,-77.474584,37.531050,6187358,203056,66600,< Richmond VA >
4,13,-71.017892,42.336029,6016425,574283,161400,< Boston MA >


The reliable P-median objective aims to minimise of the weighted sum of the **operating cost** and the **expected failure cost**. Let consider a set of **Customers** ($I$), and a set of potential location we called **Facilities** ($J$). The set of **Facilities** ($J$) is devided in two, one is the set of **nonfailable facility** ($NF$) and the set of facilities that may fail ($F$), $NF \cup F = J$. Each *customer* node has an associated **demande** ($h_i$). The cost **shiping cost** ($d_{ij}$) represent to deliver a unit of demande from a *facility* to a *customer*. Each customer applies an **penalty** ($\theta_i$) for each unit of demande that is not serve. 

The **penalty** ($\theta_i$) is incure if all facilites have failled are if the value of that penalty is smaller than the **shiping cost** ($d_{ij}$) to any of the existing facilities. To model this, we add an **emergency facility** ($u \in NF$) that is nonfailiable and as a transportation cost equals to the **penalty**.

Each facility as an expected **failure pobability** ($q_j$).

In [20]:
#P-Median Problem
P=5

# Sets of Customers abd Facilities
I  = Customers  = range(len(df.index))
J  = Facilities = df.index
F  = []
NF = []

# Sphiping cost
h = demande = []
d = shiping_cost = []

# Penalty for not shiping
theta = penalty = []

# Probability of failure
q = failure_probability = []

Similarly to UFLP there is two set of decision variables: **locations variables** ($X$) and **assignements variables** ($Y$)
$$
X_j =\left\{
        \begin{array}{ll}
          1, \text{if a facility is open}\\
          0, \text{otherwise}\\
        \end{array}
      \right.
$$

$$
Y_{ijr} =\left\{
        \begin{array}{ll}
          1, \text{if demand node i is assignement to j as a level r-assignement}\\
          0, \text{otherwise}\\
        \end{array}
      \right.
$$

In [21]:
from docplex.mp.model import Model
###################
# create one model instance
m = Model(name="Reliable P-Median problem")

###################
# Define variables

# x(j) equals 1 if node j in the solution
X = m.binary_var_dict([(j) 
                      for j in J], name="X")
# y(j,j,r) equales 1 if node j is in the solution
Y = m.binary_var_dict([(i,j,r) 
                      for i in I
                      for j in J
                      for r in range(P-1)], name="Y")

The objective $w_1$ mesure the operation cost related to the P-median problem. The objective $w_2$ compute the expected cost of failure.

$$
w_1 = \sum_{i \in I} \sum_{j \in J} h_i d_{ij} Y_{ij0}
$$

$$
w_2 = \sum_{i \in I} h_i [ \sum_{j \in NF} \sum_{r=0}^{p-1} d_{ij} q^{r} Y_{ijr} + \sum_{j \in F} \sum_{r=0}^{p-1} d_{ij} (1-q) Y_{ijr} ]
$$

In [25]:
###################
# Define Objective
w1 = m.sum(h[i] * d[i][j] * Y[i,j,0] for i in I for j in J)
w2 = m.sum( h[i] \
           * (m.sum(d[i][j] * q[i] * Y[i,j,r] for j in NF for r in range(P-1)) \
            + m.sum(d[i][j] * (1 - q[i]) * Y[i,j,r]  in F for r in range(P-1))) \
           for i in I)

IndexError: list index out of range